# Лекция 7. Построение кластера MongoDB

## Основные компоненты кластера

До сих пор мы использовали MongoDB как одиночный сервер, то есть каждый экземпляр mongod управлял полной копией данных приложения. 

Даже в случае репликации каждая реплика содержит полную копию всех данных – такую же, как на любой другой реплике. Для большинства приложений хранение всего набора данных на одном
сервере вполне приемлемо. 

Но по мере того как объем данных растет, приложение начинает предъявлять более высокие требования к пропускной способности по чтению и записи, и мощности стандартных серверов перестает хватать. В этой ситуации возникает потребность построения кластера.


Рассмотрим основные компоненты кластера.

<img src="ris1.png">

Сегментированный кластер состоит из сегментов, маршрутизаторов mongos и конфигурационных серверов. 

**Сегменты**

Сегментированный кластер MongoDB распределяет данные по одному или нескольким сегментам. Каждый сегмент развертывается в виде набора реплик, на котором хранится некоторая часть всего множества данных. Поскольку сегмент представляет собой набор реплик, то он поддерживает автоматическую отработку отказов. К отдельному сегменту можно подключаться так же, как к автономному набору реплик. Но если мы подключитесь к одному набору реплик, то
будете видеть только часть всех данных кластера.

**Маршрутизаторы**

Поскольку каждый сегмент содержит лишь часть данных кластера, то необходим интерфейс для обращения к кластеру как к единому целому. Эту роль выполняют процессы mongos. Каждый такой
процесс – это маршрутизатор, перенаправляющий все операции чтения и записи подходящему сегменту. За счет этого mongos предоставляет клиентам единый взгляд на систему в целом.

Процессы mongos потребляют мало ресурсов и не отвечают за сохранение данных. Приложение локально соединяется с mongos , а mongos уже управляет соединениями с конкретными сегментами.

**Конфигурационные серверы**

Если процессы mongos не сохраняют данные, то должен быть какой-то компонент, отвечающий за долговременное хранение  состояния сегментированного кластера; этим занимаются
конфигурационные серверы, которые хранят метаданные кластера.

В состав этих данных входят: глобальная конфигурация кластера, местоположение каждой базы данных, коллекции и диапазоны хранящихся в них данных, а также журнал изменений, в котором сохраняется история миграции данных между сегментами.

Метаданные, хранящиеся на конфигурационных серверах, играют важнейшую роль для правильного функционирования и обслуживания кластера. Например, при каждом запуске процесс mongos
получает копию метаданных от конфигурационных серверов. Без этих данных невозможно обеспечить согласованное представление сегментированного кластера. Таким образом, важность этих данных диктует стратегию проектирования и развертывания конфигурационных серверов.

В любой производственной системе, где применяется сегментирование, должно быть ровно три конфигурационных сервера, причем для обеспечения резервирования они должны размещаться на разных машинах. Для отправки им данных из процесса mongos применяется протокол двухфазной фиксации, который гарантирует согласованность всех конфигурационных серверов.

## Операции сегментирования

Кластер MongoDB распределяет данные между сегментами на двух уровнях:
- на уровне базы данных;
- на уровне коллекции.



Распределение на уровне базы данных заключается в том, что одна база данных будет вся содержаться в одном сегменте (каждой создаваемой в кластере базе данных назначается свой сегмент). 

Сегментирование на уровне коллекции позволяет содержать разные коллекции в разных сегментах.

В MongoDB сегментирование производится по диапазону. Это означает, что для каждого документа в сегментированной коллекции определяется, в какой диапазон попадает значение заданного ключа.
Чтобы отнести документ к конкретному диапазону, в MongoDB используется так называемый **сегментный ключ (shard key)**. Для создания сегментного ключа можно использовать "_id", однако лучше брать комбинацию полей.  

Еще одним важным понятием является  понятие **порции** (chunk). Порция – это непрерывный диапазон значений сегментных ключей, находящихся в одном сегменте.

Важная характеристика порций – это то, что они не физические, а логические. Другими словами, порции не соответствуют последовательность соседних документов на диске.

В основе механизма сегментирования лежит **расщепление и миграция порций**.

Сначала рассмотрим идею расщепления порций. Сразу после настройки сегментированного кластера существует всего одна порция.
Ее диапазон охватывает всю сегментированную коллекцию. 

Порция расщепляется по достижению определенного порогового размера. По умолчанию максимальный размер порции составляет 64 МБ или 100 000 документов, в зависимости от того, что будет достигнуто раньше. По мере добавления данных в новый сегментированный кластер первоначальная порция в конце концов достигает того или другого порога, после чего происходит ее расщепление. Расщепление – простая операция; по существу, она сводится к разбиению исходного диапазона на два поддиапазона и созданию двух новых порций с одинаковым количеством документов в каждой. Расщепление порции – это логическая операция.
MongoDB просто изменяет метаданные, так что из одной порции получается две. Поэтому расщепление не оказывает влияния на физическое расположение документов в сегментированной коллекции, а,
значит, происходит быстро.

Одна из самых трудных проблем при проектировании сегментированной системы – обеспечить равномерное балансирование данных. Кластеры MongoDB достигают этого за счет перемещения порций между сегментами. Эта операция называется
миграцией и, в отличие, от расщепления она подразумевает физические действия.

Миграциями управляет процесс, который называется балансировщиком. Его задача заключается в том, чтобы обеспечить равномерное
распределение данных между сегментами. Чтобы ее решить, балансировщик учитывает количество порций в каждом сегменте. 

Эвристические правила в какой-то мере зависят от полного размера данных, но обычно балансирование производится, когда разность между сегментом с наибольшим количеством порций и сегментом с наименьшим количеством порций оказывается больше восьми. В процессе балансирования порции перемещаются с сегмента, где их больше, на сегмент, где их меньше, пока количество порций не станет примерно одинаковым.

## Создание сегментированного кластера


Сегментированный кластер настраивается в два этапа. На первом запускаются все необходимые процессы mongod и mongos. На втором, более простом, выполняются команды инициализации кластера. Наш кластер будет состоять из двух сегментов и трех конфигурационных серверов. При этом мы запустим всего один процесс mongos для взаимодействия с кластером. На рисунке представлены все рабочие процессы, и для каждого в скобках указан номер порта.

<img src="ris2.png">

**Инициализация компонентов кластера**

> mkdir /data/a1

> mkdir /data/a2

> mkdir /data/a3

> mkdir /data/b1

> mkdir /data/b2

> mkdir /data/b3

Теперь запустим процессы mongod:

> mongod --shardsvr --replSet shard-a --dbpath ~/data/a1 --port 30000 --logpath ~/data/a1.log --nojournal

> mongod --shardsvr --replSet shard-a --dbpath ~/data/a2 --port 30001 --logpath ~/data/a2.log --nojournal

> mongod --shardsvr --replSet shard-a --dbpath ~/data/a3 --port 30002 --logpath ~/data/a3.log --nojournal



Также запускаем второй набор реплик:

> mongod --shardsvr --replSet shard-b --dbpath ~/data/b1 --port 30100 --logpath ~/data/b1.log --nojournal

> mongod --shardsvr --replSet shard-b --dbpath ~/data/b2 --port 30101 --logpath ~/data/b2.log --nojournal

> mongod --shardsvr --replSet shard-b --dbpath ~/data/b3 --port 30102 --logpath ~/data/b3.log --nojournal

Наборы реплик необходимо инициализировать, для этого необходимо подключиться к каждому по очереди и выполнить команды:

$ mongo localhost:30000

> rs.initiate()

После того, как данный сервер станет первичным, следует добавить остальные узлы:

> rs.add("localhost:30001")

> rs.addArb("localhost:30002")

Второй набор реплик инициализируется аналогично.

$ mongo localhost:30100

> rs.initiate()

> rs.add("localhost:30101")

> rs.addArb("localhost:30102")

Оба набора реплик можно проверить командой rs.status().

Если на данном этапе все заработало, можно переходить к запуску конфигурационных серверов. Создадим для каждого сервера отдельный каталог, и запустим по одному процессу mongod с параметром configsvr:


$ mkdir ~/data/config-1

$ mongod --configsvr --replSet configrs --dbpath ~/data/config-1 --port 27019 --bind_ip localhost

$ mkdir ~/data/config-2

$  mongod --configsvr --replSet configrs --dbpath ~/data/config-2 --port 27020 --bind_ip localhost

$ mkdir ~/data/config-3

$  mongod --configsvr --replSet configrs --dbpath ~/data/config-3 --port 27021 --bind_ip localhost

Зададим настройку реплики

$ mongo --port 27019

> rs.initiate( { _id: "configrs", members: [ { _id : 0, host : "localhost:27019" }, { _id : 1, host : "localhost:27020" }, { _id : 2, host : "localhost:27021" } ] } )

Теперь можно запускать mongos маршрутизатор. При его запуске следует указать параметр configdb,
значением которого является список адресов конфигурационных баз данных через запятую (или главный сервер реплики):

$ mongos --configdb configrs/localhost:27019 --bind_ip localhost --port 40000


**Настройка конфигурации**

Теперь все компоненты сегментированного кластера на месте и можно заняться его конфигурированием. Начнем с процесса mongos.
Чтобы упростить задачу, воспользуемся вспомогательными методами сегментирования. Все они вызываются от имени глобального объекта sh. Чтобы вывести список имеющихся методов, выполните команду sh.help().

Первым делом подключаемся к маршрутизатору и прописываем шарды:

$ mongo --port 40000

> sh.addShard("shard-a/localhost:30000")

> sh.addShard("shard-a/localhost:30001")

> sh.addShard("shard-b/localhost:30100")

> sh.addShard("shard-b/localhost:30101")

Чтобы посмотреть настройку шардирования, можно использовать следующие команды:

> db.getSiblingDB("config").shards.find()

или более кратко:

> use admin

> db.runCommand({listshards: 1})

Более подробную информацию можно получить командой:

> sh.status()

Важным шагом является разрешенние на сегментацию базы данных. Это обязательное условие для сегментирования любой коллекции. 
Назовем нашу базу данных rebus, тогда команда разрешения сегментации выглядит так:

> sh.enableSharding("rebus")

В конфигурационной базе данных имеется коллекция databases , содержащая список баз данных. В каждом документе из этого списка указано местоположение основного сегмента базы и признак partitioned , говорящий о том, сегментирована она или нет.

> use config

> db.databases.find()

{ "_id" : "rebus", "primary" : "shard-b", "partitioned" : true }

Теперь можно сегментировать какую-либо коллекцию. При этом определяется сегментный ключ, например, можно использовать составной или простой ключ. Если мы хотим, чтобы ключ  был хэширован, можно вместо этого использовать параметр {"myshardkey":"hashed"}:

> sh.shardCollection("rebus.mycollect", {"_id":"hashed"})

Чтобы посмотреть информацию о шардировании, можно использовать команду:

> db.printShardingStatus()


После того как коллекция сегментирована, кластер наконец-то готов к работе. Если теперь мы начнем писать в кластер, то данные будут распределяться по сегментам.

## Запись в сегментированный кластер

Будем писать в сегментированную коллекцию и наблюдать за формированием и перемещением порций, что и составляет суть сегментирования в MongoDB.


Можно попробовать добавлять документы прямо в консоле:

> use rebus

> for (var i=0; i<210000; i++) { db.mycollect.insert({name: "Max", amount:Math.random()*100})}

Теперь можно посмотреть, что произошло с кластером. Для этого можно переключиться на базу данных config и запросить количество порций:

> use config

> db.chunks.count()

Рассмотрим количество документов, которые были добавлены:

> db.mycollect.count()

Рассмотрим объем памяти, занятый данными:

> db.mycollect.stats().size

Для получения более детальной информации вызовем метод sh.status(), который напечатает все порции с указанием их диапазонов.

> sh.status()


Чтобы посмотреть, сколько порций выделено в каждом сегменте, можно использовать команды:

> db.chunks.count({"shard": "shard-a"})

> db.chunks.count({"shard": "shard-b"})